<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# Generating C Code to implement Method of Lines Timestepping for Explicit Runge Kutta Methods

## Authors: Zach Etienne & Brandon Clark

## This tutorial notebook generates three blocks of C Code in order to perform Method of Lines timestepping. 

**Notebook Status:** <font color='green'><b> Validated </b></font>

**Validation Notes:** This tutorial notebook has been confirmed to be self-consistent with its corresponding NRPy+ module, as documented [below](#code_validation). All Runge-Kutta Butcher tables were validated using truncated Taylor series in [a separate module](Tutorial-RK_Butcher_Table_Validation.ipynb). Finally, C-code implementation of RK4 was validated against a trusted version. C-code implementations of other RK methods seem to work as expected in the context of solving the scalar wave equation in Cartesian coordinates.

### NRPy+ Source Code for this module: 
* [MoLtimestepping/C_Code_Generation.py](../edit/MoLtimestepping/C_Code_Generation.py)
* [MoLtimestepping/RK_Butcher_Table_Dictionary.py](../edit/MoLtimestepping/RK_Butcher_Table_Dictionary.py) ([**Tutorial**](Tutorial-RK_Butcher_Table_Dictionary.ipynb)) Stores the Butcher tables for the explicit Runge Kutta methods

## Introduction:

When numerically solving a partial differential equation initial-value problem, subject to suitable boundary conditions,  we implement Method of Lines to "integrate" the solution forward in time.


### The Method of Lines:

Once we have the initial data for a PDE, we "evolve it forward in time", using the [Method of Lines](https://reference.wolfram.com/language/tutorial/NDSolveMethodOfLines.html). In short, the Method of Lines enables us to handle 
1. the **spatial derivatives** of an initial value problem PDE using **standard finite difference approaches**, and
2. the **temporal derivatives** of an initial value problem PDE using **standard strategies for solving ordinary differential equations (ODEs), like Runge Kutta methods** so long as the initial value problem PDE can be written in the first-order-in-time form
$$\partial_t \vec{f} = \mathbf{M}\ \vec{f},$$
where $\mathbf{M}$ is an $N\times N$ matrix containing only *spatial* differential operators that act on the $N$-element column vector $\vec{f}$. $\mathbf{M}$ may not contain $t$ or time derivatives explicitly; only *spatial* partial derivatives are allowed to appear inside $\mathbf{M}$.

You may find the next module [Tutorial-ScalarWave](Tutorial-ScalarWave.ipynb) extremely helpful as an example for implementing the Method of Lines for solving the Scalar Wave equation in Cartesian coordinates.

### Generating the C code:
This module describes how three C code blocks are written to implement Method of Lines timestepping for a specified RK method. The first block is dedicated to allocating memory for the appropriate number of grid function lists needed for the given RK method. The second block will implement the Runge Kutta numerical scheme based on the corresponding Butcher table. The third block will free up the previously allocated memory after the Method of Lines run is complete. These blocks of code are stored within the following three header files respectively

1. `MoLtimestepping/RK_Allocate_Memory.h`
1. `MoLtimestepping/RK_MoL.h`
1. `MoLtimestepping/RK_Free_Memory.h`

The generated code is then included in future Start-to-Finish example tutorial notebooks when solving PDEs numerically.

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

This notebook is organized as follows

1. [Step 1](#initializenrpy): Initialize needed Python/NRPy+ modules
1. [Step 2](#diagonal): Checking if Butcher Table is Diagonal
1. [Step 3](#ccode): Generating the C Code
    1. [Step 3.a](#allocate): Allocating Memory, `MoLtimestepping/RK_Allocate_Memory.h`
    1. [Step 3.b](#rkmol): Implementing the Runge Kutta Scheme for Method of Lines Timestepping,  `MoLtimestepping/RK_MoL.h`
    1. [Step 3.c](#free): Freeing Allocated Memory, `MoLtimestepping/RK_Free_Memory.h`
1. [Step 4](#code_validation): Code Validation against `MoLtimestepping.RK_Butcher_Table_Generating_C_Code` NRPy+ module 
1. [Step 5](#latex_pdf_output): Output this notebook to $\LaTeX$-formatted PDF file

<a id='initializenrpy'></a>

# Step 1: Initialize needed Python/NRPy+ modules [Back to [top](#toc)\]
$$\label{initializenrpy}$$

Let's start by importing all the needed modules from Python/NRPy+:

In [1]:
import sympy as sp
import NRPy_param_funcs as par
from MoLtimestepping.RK_Butcher_Table_Dictionary import Butcher_dict

<a id='diagonal'></a>

# Step 2: Checking if a Butcher table is Diagonal [Back to [top](#toc)\]
$$\label{diagonal}$$

A diagonal Butcher table takes the form 

$$\begin{array}{c|cccccc}
    0 & \\
    a_1 & a_1 & \\ 
    a_2 & 0 & a_2 & \\
    a_3 & 0 & 0 & a_3 & \\ 
    \vdots & \vdots & \ddots & \ddots & \ddots \\ 
    a_s & 0 & 0 & 0 & \cdots & a_s \\ \hline
     & b_1 & b_2 & b_3 & \cdots & b_{s-1} & b_s
\end{array}$$

where $s$ is the number of required predictor-corrector steps for a given RK method (see [Butcher, John C. (2008)](https://onlinelibrary.wiley.com/doi/book/10.1002/9780470753767)). One known diagonal RK method is the classic RK4 represented in Butcher table form as:

$$\begin{array}{c|cccc}
    0 & \\
    1/2 & 1/2 & \\ 
    1/2 & 0 & 1/2 & \\
    1 & 0 & 0 & 1 & \\ \hline
     & 1/6 & 1/3 & 1/3 & 1/6
\end{array} $$

Diagonal Butcher tables are nice when it comes to saving required memory space. Each new step for a diagonal RK method, when computing the new $k_i$, does not depend on the previous calculation, and so there are ways to save memory. Signifcantly so in large three-dimensional spatial grid spaces.

In [2]:
def diagonal(key):
    diagonal = True #  Start with the Butcher table is diagonal
    Butcher = Butcher_dict[key][0]
    L = len(Butcher)-1 # Establish the number of rows to check for diagonal trait, all bust last row
    row_idx = 0 # Initialize the Butcher table row index
    for i in range(L): # Check all the desired rows
        for j in range(1,row_idx): # Check each element before the diagonal element in a row
            if Butcher[i][j] != sp.sympify(0): # If any element is non-zero, then the table is not diagonal
                diagonal = False
                break
        row_idx += 1 # Update to check the next row
    return diagonal

# State whether each Butcher table is diagonal or not
for key, value in Butcher_dict.items():
    if diagonal(key) == True:
        print("The RK method "+str(key)+" is diagonal!")
    else:
        print("The RK method "+str(key)+" is NOT diagonal!")

The RK method Euler is diagonal!
The RK method RK2 Heun is diagonal!
The RK method RK2 MP is diagonal!
The RK method RK2 Ralston is diagonal!
The RK method RK3 is NOT diagonal!
The RK method RK3 Heun is diagonal!
The RK method RK3 Ralston is diagonal!
The RK method SSPRK3 is NOT diagonal!
The RK method RK4 is diagonal!
The RK method DP5 is NOT diagonal!
The RK method DP5alt is NOT diagonal!
The RK method CK5 is NOT diagonal!
The RK method DP6 is NOT diagonal!
The RK method L6 is NOT diagonal!
The RK method DP8 is NOT diagonal!


<a id='ccode'></a>

# Step 3: Generating the C Code [Back to [top](#toc)\]
$$\label{ccode}$$

The following sections build up the C code for implementing the Method of Lines timestepping algorithm for solving PDEs. To see what the C code looks like for a particular method, simply change the `RK_method` below, otherwise it will default to `"RK4"`. 

<a id='allocate'></a>

## Step 3.a: Allocating Memory, `MoLtimestepping/RK_Allocate_Memory.h`  [Back to [top](#toc)\]
$$\label{allocate}$$

We define the function `RK_Allocate()` which generates the C code for allocating the memory for the appropriate number of grid function lists given a Runge Kutta method. The function writes the C code to the header file `MoLtimestepping/RK_Allocate_Memory.h`.

In [3]:
# Choose a method to see the C code print out for
RK_method = "RK3 Ralston"

In [4]:
def RK_Allocate(RK_method="RK4"):
    with open("MoLtimestepping/RK_Allocate_Memory"+str(RK_method).replace(" ", "_")+".h", "w") as file:
        file.write("// Code snippet allocating gridfunction memory for \""+str(RK_method)+"\" method:\n")
        # No matter the method we define gridfunctions "y_n_gfs" to store the initial data    
        file.write("REAL *restrict y_n_gfs = (REAL *)malloc(sizeof(REAL) * NUM_EVOL_GFS * Nxx_plus_2NGHOSTS_tot);\n")
        if diagonal(RK_method) == True and "RK3" in RK_method:
            file.write("""REAL *restrict k1_or_y_nplus_a21_k1_or_y_nplus1_running_total_gfs = (REAL *)malloc(sizeof(REAL) * NUM_EVOL_GFS * Nxx_plus_2NGHOSTS_tot);
REAL *restrict k2_or_y_nplus_a32_k2_gfs = (REAL *)malloc(sizeof(REAL) * NUM_EVOL_GFS * Nxx_plus_2NGHOSTS_tot);
REAL *restrict diagnostic_output_gfs = k1_or_y_nplus_a21_k1_or_y_nplus1_running_total_gfs;""")
        else:    
            if diagonal(RK_method) == False: #  Allocate memory for non-diagonal Butcher tables 
                # Determine the number of k_i steps based on length of Butcher Table
                num_k = len(Butcher_dict[RK_method][0])-1
                # For non-diagonal tables an intermediate gridfunction "next_y_input" is needed for rhs evaluations
                file.write("REAL *restrict next_y_input_gfs = (REAL *)malloc(sizeof(REAL) * NUM_EVOL_GFS * Nxx_plus_2NGHOSTS_tot);\n")
                for i in range(num_k): # Need to allocate all k_i steps for a given method 
                    file.write("REAL *restrict k"+str(i+1)+"_gfs = (REAL *)malloc(sizeof(REAL) * NUM_EVOL_GFS * Nxx_plus_2NGHOSTS_tot);\n")
                file.write("REAL *restrict diagnostic_output_gfs = k1_gfs;\n")
            else: # Allocate memory for diagonal Butcher tables, which use a "y_nplus1_running_total gridfunction"
                file.write("REAL *restrict y_nplus1_running_total_gfs = (REAL *)malloc(sizeof(REAL) * NUM_EVOL_GFS * Nxx_plus_2NGHOSTS_tot);\n")               
                if RK_method != 'Euler': # Allocate memory for diagonal Butcher tables that aren't Euler
                    # Need k_odd for k_1,3,5... and k_even for k_2,4,6...
                    file.write("REAL *restrict k_odd_gfs = (REAL *)malloc(sizeof(REAL) * NUM_EVOL_GFS * Nxx_plus_2NGHOSTS_tot);\n")
                    file.write("REAL *restrict k_even_gfs = (REAL *)malloc(sizeof(REAL) * NUM_EVOL_GFS * Nxx_plus_2NGHOSTS_tot);\n")
                file.write("REAL *restrict diagnostic_output_gfs = y_nplus1_running_total_gfs;\n")

RK_Allocate(RK_method)
print("This is the memory allocation C code for the "+str(RK_method)+" method: \n")
with open("MoLtimestepping/RK_Allocate_Memory"+str(RK_method).replace(" ", "_")+".h", "r") as file:
    print(file.read())

This is the memory allocation C code for the RK3 Ralston method: 

// Code snippet allocating gridfunction memory for "RK3 Ralston" method:
REAL *restrict y_n_gfs = (REAL *)malloc(sizeof(REAL) * NUM_EVOL_GFS * Nxx_plus_2NGHOSTS_tot);
REAL *restrict k1_or_y_nplus_a21_k1_or_y_nplus1_running_total_gfs = (REAL *)malloc(sizeof(REAL) * NUM_EVOL_GFS * Nxx_plus_2NGHOSTS_tot);
REAL *restrict k2_or_y_nplus_a32_k2_gfs = (REAL *)malloc(sizeof(REAL) * NUM_EVOL_GFS * Nxx_plus_2NGHOSTS_tot);
REAL *restrict diagnostic_output_gfs = k1_or_y_nplus_a21_k1_or_y_nplus1_running_total_gfs;


<a id='rkmol'></a>

## Step 3.b: Implementing the Runge Kutta Scheme for Method of Lines Timestepping,  `MoLtimestepping/RK_MoL.h` [Back to [top](#toc)\]
$$\label{rkmol}$$

We define the function `RK_MoL()` which generates the C code for implementing Method of Lines using a specified Runge Kutta scheme. The function writes the C code to the header file `MoLtimestepping/RK_MoL.h`.

In [5]:
def RK_MoL(RK_method,RHS_string, post_RHS_string):
    Butcher = Butcher_dict[RK_method][0] # Get the desired Butcher table from the dictionary
    num_steps = len(Butcher)-1 # Specify the number of required steps to update solution
    indent = "  "
    with open("MoLtimestepping/RK_MoL"+str(RK_method).replace(" ", "_")+".h", "w") as file:
        file.write("// Code snippet implementing "+RK_method+" algorithm for Method of Lines timestepping\n")
        # Diagonal RK3 only!!!
        if diagonal(RK_method) == True and "RK3" in RK_method:
            #  In a diagonal RK3 method, only 3 gridfunctions need be defined. Below implements this approach.
            file.write("""
// In a diagonal RK3 method like this one, only 3 gridfunctions need be defined. Below implements this approach.
// Using y_n_gfs as input, compute k1 and apply boundary conditions
"""+RHS_string.replace("RK_INPUT_GFS" ,"y_n_gfs").
               replace("RK_OUTPUT_GFS","k1_or_y_nplus_a21_k1_or_y_nplus1_running_total_gfs")+"""
LOOP_ALL_GFS_GPS(i) {
    // Store k1_or_y_nplus_a21_k1_or_y_nplus1_running_total_gfs now as
    //  the update for the next rhs evaluation y_n + a21*k1*dt:
    k1_or_y_nplus_a21_k1_or_y_nplus1_running_total_gfs[i] = ("""+sp.ccode(Butcher[1][1]).replace("L","")+""")*k1_or_y_nplus_a21_k1_or_y_nplus1_running_total_gfs[i]*dt + y_n_gfs[i];
}
// Apply boundary conditions to y_n + a21*k1*dt:
"""+post_RHS_string.replace("RK_OUTPUT_GFS","k1_or_y_nplus_a21_k1_or_y_nplus1_running_total_gfs")+"""

// Compute k2 using yn + a21*k1*dt
"""+RHS_string.replace("RK_INPUT_GFS" ,"k1_or_y_nplus_a21_k1_or_y_nplus1_running_total_gfs").
               replace("RK_OUTPUT_GFS","k2_or_y_nplus_a32_k2_gfs")+"""
LOOP_ALL_GFS_GPS(i) {
    // Reassign k1_or_y_nplus_a21_k1_or_y_nplus1_running_total_gfs to be
    //    the running total y_{n+1}
    k1_or_y_nplus_a21_k1_or_y_nplus1_running_total_gfs[i] = ("""+sp.ccode(Butcher[3][1]).replace("L","")+""")*(k1_or_y_nplus_a21_k1_or_y_nplus1_running_total_gfs[i] - y_n_gfs[i])/("""+sp.ccode(Butcher[1][1]).replace("L","")+""") + y_n_gfs[i];

    // Add a32*k2*dt to the running total
    k1_or_y_nplus_a21_k1_or_y_nplus1_running_total_gfs[i]+= ("""+sp.ccode(Butcher[3][2]).replace("L","")+""")*k2_or_y_nplus_a32_k2_gfs[i]*dt;

    // Store k2_or_y_nplus_a32_k2_gfs now as y_n + a32*k2*dt
    k2_or_y_nplus_a32_k2_gfs[i] = ("""+sp.ccode(Butcher[2][2]).replace("L","")+""")*k2_or_y_nplus_a32_k2_gfs[i]*dt + y_n_gfs[i];
}
// Apply boundary conditions to both y_n + a32*k2 (stored in k2_or_y_nplus_a32_k2_gfs)
//    ... and the y_{n+1} running total, as they have not been applied yet to k2-related gridfunctions:
"""+post_RHS_string.replace("RK_OUTPUT_GFS","k2_or_y_nplus_a32_k2_gfs")+"""
"""+post_RHS_string.replace("RK_OUTPUT_GFS","k1_or_y_nplus_a21_k1_or_y_nplus1_running_total_gfs")+"""

// Compute k3
"""+RHS_string.replace("RK_INPUT_GFS" ,"k2_or_y_nplus_a32_k2_gfs").
               replace("RK_OUTPUT_GFS","y_n_gfs")+"""
LOOP_ALL_GFS_GPS(i) {
    // Add k3 to the running total and save to y_n
    y_n_gfs[i] = k1_or_y_nplus_a21_k1_or_y_nplus1_running_total_gfs[i] + ("""+sp.ccode(Butcher[3][3]).replace("L","")+""")*y_n_gfs[i]*dt;
}
// Apply boundary conditions to the running total
"""+post_RHS_string.replace("RK_OUTPUT_GFS","y_n_gfs")+"\n")
        else:    
            y_n           = "y_n_gfs"
            if diagonal(RK_method) == False:
                for s in range(num_steps):
                    next_y_input  = "next_y_input_gfs"

                    # If we're on the first step (s=0), we use y_n gridfunction as input. 
                    #      Otherwise next_y_input is input. Output is just the reverse.
                    if s==0: # If on first step:
                        file.write(RHS_string.replace("RK_INPUT_GFS",y_n).replace("RK_OUTPUT_GFS","k"+str(s+1)+"_gfs")+"\n")
                    else:    # If on second step or later:
                        file.write(RHS_string.replace("RK_INPUT_GFS",next_y_input).replace("RK_OUTPUT_GFS","k"+str(s+1)+"_gfs")+"\n")
                    file.write("LOOP_ALL_GFS_GPS(i) {\n")
                    RK_update_string = ""
                    if s == num_steps-1: # If on final step:
                        RK_update_string += indent + y_n+"[i] += dt*("
                    else:                # If on anything but the final step:
                        RK_update_string += indent + next_y_input+"[i] = "+y_n+"[i] + dt*("
                    for m in range(s+1):
                        if Butcher[s+1][m+1] != 0:
                            if Butcher[s+1][m+1] != 1:
                                RK_update_string += " + k"+str(m+1)+"_gfs[i]*("+sp.ccode(Butcher[s+1][m+1]).replace("L","")+")"
                            else:
                                RK_update_string += " + k"+str(m+1)+"_gfs[i]"
                    RK_update_string += " );\n}\n"
                    file.write(RK_update_string)
                    if s == num_steps-1: # If on final step:
                        file.write(post_RHS_string.replace("RK_OUTPUT_GFS",y_n)+"\n")
                    else:                # If on anything but the final step:
                        file.write(post_RHS_string.replace("RK_OUTPUT_GFS",next_y_input)+"\n")
            else:
                y_nplus1_running_total = "y_nplus1_running_total_gfs"
                if RK_method == 'Euler': # Euler's method doesn't require any k_i, and gets its own unique algorithm
                    file.write(RHS_string.replace("RK_INPUT_GFS",y_n).replace("RK_OUTPUT_GFS",y_nplus1_running_total)+"\n")
                    file.write("LOOP_ALL_GFS_GPS(i) {\n")
                    file.write(indent + y_n+"[i] +=  "+y_nplus1_running_total+"[i]*dt;\n")
                    file.write("}\n")
                    file.write(post_RHS_string.replace("RK_OUTPUT_GFS",y_n)+"\n")
                else:
                    for s in range(num_steps):
                        # If we're on the first step (s=0), we use y_n gridfunction as input. 
                        # and k_odd as output.
                        if s == 0:
                            rhs_input  = "y_n_gfs"
                            rhs_output = "k_odd_gfs"
                        # For the remaining steps the inputs and ouputs alternate between k_odd and k_even
                        elif s%2 == 0:
                            rhs_input = "k_even_gfs"
                            rhs_output = "k_odd_gfs"
                        else:
                            rhs_input = "k_odd_gfs"
                            rhs_output = "k_even_gfs"
                        file.write(RHS_string.replace("RK_INPUT_GFS",rhs_input).replace("RK_OUTPUT_GFS",rhs_output)+"\n")
                        file.write("LOOP_ALL_GFS_GPS(i) {\n")
                        if s == num_steps-1: # If on the final step
                            if Butcher[num_steps][s+1] !=0:
                                if Butcher[num_steps][s+1] !=1:  
                                    file.write(indent+y_n+"[i] += "+y_nplus1_running_total+"[i] + "+rhs_output+"[i]*dt*("+sp.ccode(Butcher[num_steps][s+1]).replace("L","")+");\n")
                                else: 
                                    file.write(indent+y_n+"[i] += "+y_nplus1_running_total+"[i] + "+rhs_output+"[i]*dt;\n")     
                            file.write("}\n")
                            file.write(post_RHS_string.replace("RK_OUTPUT_GFS",y_n)+"\n")
                        else: # For anything besides the final step
                            if s == 0:
                                file.write(indent+y_nplus1_running_total+"[i] = "+rhs_output+"[i]*dt*("+sp.ccode(Butcher[num_steps][s+1]).replace("L","")+");\n")
                                file.write(indent+rhs_output+"[i] = "+y_n+"[i] + "+rhs_output+"[i]*dt*("+sp.ccode(Butcher[s+1][s+1]).replace("L","")+");\n")
                            else:
                                if Butcher[num_steps][s+1] !=0:
                                    if Butcher[num_steps][s+1] !=1:
                                        file.write(indent+y_nplus1_running_total+"[i] += "+rhs_output+"[i]*dt*("+sp.ccode(Butcher[num_steps][s+1]).replace("L","")+");\n")
                                    else: 
                                        file.write(indent+y_nplus1_running_total+"[i] += "+rhs_output+"[i]*dt;\n")
                                if Butcher[s+1][s+1] !=0:
                                    if Butcher[s+1][s+1] !=1:
                                        file.write(indent+rhs_output+"[i] = "+y_n+"[i] + "+rhs_output+"[i]*dt*("+sp.ccode(Butcher[s+1][s+1]).replace("L","")+");\n")
                                    else:
                                        file.write(indent+rhs_output+"[i] = "+y_n+"[i] + "+rhs_output+"[i]*dt;\n")
                            file.write("}\n")
                            file.write(post_RHS_string.replace("RK_OUTPUT_GFS",rhs_output)+"\n")
                        
RK_MoL(RK_method,"rhs_eval(Nxx,Nxx_plus_2NGHOSTS,dxx, RK_INPUT_GFS, RK_OUTPUT_GFS);",
      "")
print("This is the MoL timestepping RK scheme C code for the "+str(RK_method)+" method: \n")
with open("MoLtimestepping/RK_MoL"+str(RK_method).replace(" ", "_")+".h", "r") as file:
    print(file.read())

This is the MoL timestepping RK scheme C code for the RK3 Ralston method: 

// Code snippet implementing RK3 Ralston algorithm for Method of Lines timestepping

// In a diagonal RK3 method like this one, only 3 gridfunctions need be defined. Below implements this approach.
// Using y_n_gfs as input, compute k1 and apply boundary conditions
rhs_eval(Nxx,Nxx_plus_2NGHOSTS,dxx, y_n_gfs, k1_or_y_nplus_a21_k1_or_y_nplus1_running_total_gfs);
LOOP_ALL_GFS_GPS(i) {
    // Store k1_or_y_nplus_a21_k1_or_y_nplus1_running_total_gfs now as
    //  the update for the next rhs evaluation y_n + a21*k1*dt:
    k1_or_y_nplus_a21_k1_or_y_nplus1_running_total_gfs[i] = (1.0/2.0)*k1_or_y_nplus_a21_k1_or_y_nplus1_running_total_gfs[i]*dt + y_n_gfs[i];
}
// Apply boundary conditions to y_n + a21*k1*dt:


// Compute k2 using yn + a21*k1*dt
rhs_eval(Nxx,Nxx_plus_2NGHOSTS,dxx, k1_or_y_nplus_a21_k1_or_y_nplus1_running_total_gfs, k2_or_y_nplus_a32_k2_gfs);
LOOP_ALL_GFS_GPS(i) {
    // Reassign k1_or_y_nplus_a21_k1_

<a id='free'></a>

## Step 3.c: Freeing Allocated Memory, `MoLtimestepping/RK_Free_Memory.h` [Back to [top](#toc)\]
$$\label{free}$$

We define the function `RK_free()` which generates the C code for freeing the memory that was being occupied by the grid functions lists that had been allocated. The function writes the C code to the header file `MoLtimestepping/RK_Free_Memory.h`

In [6]:
def RK_free(RK_method):
    L = len(Butcher_dict[RK_method][0])-1 # Useful when freeing k_i gridfunctions

    with open("MoLtimestepping/RK_Free_Memory"+str(RK_method).replace(" ", "_")+".h", "w") as file:
        file.write("// CODE SNIPPET FOR FREEING ALL ALLOCATED MEMORY FOR "+str(RK_method)+" METHOD:\n")
        if diagonal(RK_method) == True and "RK3" in RK_method:
            file.write("""
free(k1_or_y_nplus_a21_k1_or_y_nplus1_running_total_gfs);
free(k2_or_y_nplus_a32_k2_gfs);
free(y_n_gfs);""")
        else:
            file.write("free(y_n_gfs);\n") 
            if diagonal(RK_method) == False: # Free memory for allocations made for non-diagonal cases
                file.write("free(next_y_input_gfs);\n")
                for i in range(L):
                    file.write("free(k"+str(i+1)+"_gfs);\n")
            else: # Free memory for allocations made for diagonal cases
                file.write("free(y_nplus1_running_total_gfs);\n")
                if RK_method != 'Euler':       
                    file.write("free(k_odd_gfs);\n")
                    file.write("free(k_even_gfs);\n")

RK_free(RK_method)
print("This is the freeing allocated memory C code for the "+str(RK_method)+" method: \n")
with open("MoLtimestepping/RK_Free_Memory"+str(RK_method).replace(" ", "_")+".h", "r") as file:
    print(file.read())

This is the freeing allocated memory C code for the RK3 Ralston method: 

// CODE SNIPPET FOR FREEING ALL ALLOCATED MEMORY FOR RK3 Ralston METHOD:

free(k1_or_y_nplus_a21_k1_or_y_nplus1_running_total_gfs);
free(k2_or_y_nplus_a32_k2_gfs);
free(y_n_gfs);


<a id='code_validation'></a>

# Step 4: Code Validation against `MoLtimestepping.RK_Butcher_Table_Generating_C_Code` NRPy+ module [Back to [top](#toc)\]
$$\label{code_validation}$$

As a code validation check, we verify agreement in the dictionary of Butcher tables between

1. this tutorial and 
2. the NRPy+ [MoLtimestepping.RK_Butcher_Table_Generating_C_Code](../edit/MoLtimestepping/RK_Butcher_Table_Generating_C_Code.py) module.

We generate the header files for each RK method and check for agreement with the NRPY+ module.

In [7]:
import sys
import MoLtimestepping.C_Code_Generation as MoLC

print("\n\n ### BEGIN VALIDATION TESTS ###")
import filecmp
fileprefix1 = "MoLtimestepping/RK_Allocate_Memory"
fileprefix2 = "MoLtimestepping/RK_MoL"
fileprefix3 = "MoLtimestepping/RK_Free_Memory"
for key, value in Butcher_dict.items():
    MoLC.MoL_C_Code_Generation(key, 
                                "rhs_eval(Nxx,Nxx_plus_2NGHOSTS,dxx, RK_INPUT_GFS, RK_OUTPUT_GFS);", 
                                "apply_bcs(Nxx,Nxx_plus_2NGHOSTS, RK_OUTPUT_GFS);")
    RK_Allocate(key)
    RK_MoL(key,
            "rhs_eval(Nxx,Nxx_plus_2NGHOSTS,dxx, RK_INPUT_GFS, RK_OUTPUT_GFS);", 
            "apply_bcs(Nxx,Nxx_plus_2NGHOSTS, RK_OUTPUT_GFS);")
    RK_free(key)
    if filecmp.cmp(fileprefix1+str(key).replace(" ", "_")+".h" , fileprefix1+".h") == False:
        print("VALIDATION TEST FAILED ON files: "+fileprefix1+str(key).replace(" ", "_")+".h and "+ fileprefix1+".h")
        sys.exit(1)
    elif filecmp.cmp(fileprefix2+str(key).replace(" ", "_")+".h" , fileprefix2+".h") == False:
        print("VALIDATION TEST FAILED ON files: "+fileprefix2+str(key).replace(" ", "_")+".h and "+ fileprefix2+".h")
        sys.exit(1)
    elif filecmp.cmp(fileprefix3+str(key).replace(" ", "_")+".h" , fileprefix3+".h") == False:
        print("VALIDATION TEST FAILED ON files: "+fileprefix3+str(key).replace(" ", "_")+".h and "+ fileprefix3+".h")
        sys.exit(1)
    else:
        print("VALIDATION TEST PASSED on all files from "+str(key)+" method")
print("### END VALIDATION TESTS ###")



 ### BEGIN VALIDATION TESTS ###
VALIDATION TEST PASSED on all files from Euler method
VALIDATION TEST PASSED on all files from RK2 Heun method
VALIDATION TEST PASSED on all files from RK2 MP method
VALIDATION TEST PASSED on all files from RK2 Ralston method
VALIDATION TEST PASSED on all files from RK3 method
VALIDATION TEST PASSED on all files from RK3 Heun method
VALIDATION TEST PASSED on all files from RK3 Ralston method
VALIDATION TEST PASSED on all files from SSPRK3 method
VALIDATION TEST PASSED on all files from RK4 method
VALIDATION TEST PASSED on all files from DP5 method
VALIDATION TEST PASSED on all files from DP5alt method
VALIDATION TEST PASSED on all files from CK5 method
VALIDATION TEST PASSED on all files from DP6 method
VALIDATION TEST PASSED on all files from L6 method
VALIDATION TEST PASSED on all files from DP8 method
### END VALIDATION TESTS ###


<a id='latex_pdf_output'></a>

# Step 5: Output this notebook to $\LaTeX$-formatted PDF \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

The following code cell converts this Jupyter notebook into a proper, clickable $\LaTeX$-formatted PDF file. After the cell is successfully run, the generated PDF may be found in the root NRPy+ tutorial directory, with filename
[Tutorial-RK_Butcher_Table_Generating_C_Code.pdf](Tutorial-RK_Butcher_Table_Generating_C_Code.pdf) (Note that clicking on this link may not work; you may need to open the PDF file through another means.)

In [8]:
!jupyter nbconvert --to latex --template latex_nrpy_style.tplx --log-level='WARN' Tutorial-Method_of_Lines-C_Code_Generation.ipynb
!pdflatex -interaction=batchmode Tutorial-Method_of_Lines-C_Code_Generation.tex
!pdflatex -interaction=batchmode Tutorial-Method_of_Lines-C_Code_Generation.tex
!pdflatex -interaction=batchmode Tutorial-Method_of_Lines-C_Code_Generation.tex
!rm -f Tut*.out Tut*.aux Tut*.log

This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
